<a href="https://colab.research.google.com/github/Leandro-Bernardo/Landside-monitoring-system/blob/main/Jurujuba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


#Visualização dos dados do MPU6050 e sensores de umidade



####Antes de executar o próximo passo você deve





1.   Baixar o arquivo para analisar em https://drive.google.com/drive/folders/1CGCfwibPKUVhXzCQkeY_bW6fP8x7NedE
2.   Adiconar o arquivo ao colab, para isso, veja o video https://www.youtube.com/watch?v=PtTEukfwAig&feature=youtu.be
3. Dados de chuva baixados em http://www2.cemaden.gov.br/mapainterativo/#

#### Canal dos testes pode ser visto em 
https://thingspeak.com/channels/1670099

In [1]:
#!pip install --upgrade plotly  ## É instalada a mais nova versã do Plotly Para ter gráficos interativos

### Codigo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/PDPA/Dados'

/content/drive/MyDrive/PDPA/Dados


In [3]:
#Carregar Bibliotecas
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
# import dash

In [4]:
#Pedir ao pandas que mostre graficos no plotly
pd.options.plotting.backend = "plotly"

In [5]:
#É lido o arquivo onde estão os dados
dados3 = pd.read_csv("J3.csv")
dados2 = pd.read_csv("J2.csv")
dados1 = pd.read_csv("J1.csv")
dados1 = dados1.drop(dados1.columns[[9,10,11,12]], axis=1) #Elimina colunas com informações geograficas do experimento, neste caso não fornecidas pelo thingspeak, pelo que não contem infomração relevante.
dados2 = dados2.drop(dados2.columns[[9,10,11,12]], axis=1) #Elimina colunas com informações geograficas do experimento, neste caso não fornecidas pelo thingspeak, pelo que não contem infomração relevante.
dados3 = dados3.drop(dados3.columns[[9,10,11,12]], axis=1) #Elimina colunas com informações geograficas do experimento, neste caso não fornecidas pelo thingspeak, pelo que não contem infomração relevante.

In [6]:
#Calcula o tempo
def AjustarDataframe(dados):
  dados['tempo']=pd.to_datetime(dados['created_at'])

#Modifica os nomes das columnas
  dados=dados.rename(columns={"field1":"acelX"})
  dados=dados.rename(columns={"field2":"acelY"})
  dados=dados.rename(columns={"field3":"acelZ"})
  dados=dados.rename(columns={"field4":"temperatura"})
  dados=dados.rename(columns={"field5":"U1"})
  dados=dados.rename(columns={"field6":"U2"})
  dados=dados.rename(columns={"field7":"U3"})
  dados['ntemperatura']=dados['temperatura']/100  
  dados['AceleraçãoX']=dados['acelX']*9.81/16384
  dados['AceleraçãoY']=dados['acelY']*9.81/16384
  dados['AceleraçãoZ']=dados['acelZ']*9.81/16384
  return(dados)

dados1 = AjustarDataframe(dados1)
dados2 = AjustarDataframe(dados2)
dados3 = AjustarDataframe(dados3)

In [ ]:
display(dados1)

In [8]:
def EliminaExtremos(dados):

  valelimina = 1023
  vnovo = 10
  dados.loc[dados['U1'] > valelimina,'U1']  = vnovo
  dados.loc[dados['U2'] > valelimina,'U2']  = vnovo
  dados.loc[dados['U3'] > valelimina,'U3']  = vnovo

  valelimina = -1023
  dados.loc[dados['U1'] < valelimina,'U1']  = vnovo
  dados.loc[dados['U2'] < valelimina,'U2']  = vnovo
  dados.loc[dados['U3'] < valelimina,'U3']  = vnovo

  valelimina = -1
  dados.loc[dados['U1'] == valelimina,'U1']  = vnovo
  dados.loc[dados['U2'] == valelimina,'U2']  = vnovo
  dados.loc[dados['U3'] == valelimina,'U3']  = vnovo
  
  valelimina = 0
  vnovo = math.nan
  vnovo = 0
  dados.loc[dados['U1'] == valelimina,'U1']  = vnovo
  dados.loc[dados['U2'] == valelimina,'U2']  = vnovo
  dados.loc[dados['U3'] == valelimina,'U3']  = vnovo
  return(dados)
#ndados = (dados[dados['U1']!=3653])

dados1 = EliminaExtremos(dados1)
dados2 = EliminaExtremos(dados2)
dados3 = EliminaExtremos(dados3)

In [ ]:

fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=dados1['tempo'], y=dados1['U1'], mode='markers', name='J1_U1'))
fig2.add_trace(go.Scatter(x=dados1['tempo'], y=dados1['U2'], mode='markers', name='J1_U2'))
fig2.add_trace(go.Scatter(x=dados1['tempo'], y=dados1['U3'], mode='markers', name='J1_U3'))
fig2.add_trace(go.Scatter(x=dados3['tempo'], y=dados3['U1'], mode='markers', name='J3_U1'))
fig2.add_trace(go.Scatter(x=dados3['tempo'], y=dados3['U2'], mode='markers', name='J3_U2'))
fig2.add_trace(go.Scatter(x=dados3['tempo'], y=dados3['U3'], mode='markers', name='J3_U3'))
fig2.add_trace(go.Scatter(x=dados2['tempo'], y=dados2['U1'], mode='markers', name='J2_U1'))
fig2.add_trace(go.Scatter(x=dados2['tempo'], y=dados2['U2'], mode='markers', name='J2_U2'))
fig2.add_trace(go.Scatter(x=dados2['tempo'], y=dados2['U3'], mode='markers', name='J2_U3'))
#fig2['data'][0]['line']['color']='rgb(255,0,0)'

#Códigos para melhorar o grafico
fig2.update_layout(template="plotly_white")
fig2.update_traces(line=dict(width=2)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (ºC)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### Análise de Dados

#### Contagem de Dados

In [11]:
dados1count = dados1.drop(['entry_id','acelX','acelY','acelZ','temperatura','tempo','ntemperatura','AceleraçãoX','AceleraçãoY','AceleraçãoZ'], axis = 1)
amostras1 = len(dados1count['U1']) +len(dados1count['U2']) + len(dados1count['U3'])
print(f'J1 tem {amostras1} dados')

J1 tem 11298 dados


In [12]:
dados2count = dados2.drop(['entry_id','acelX','acelY','acelZ','temperatura','tempo','ntemperatura','AceleraçãoX','AceleraçãoY','AceleraçãoZ'], axis = 1)
amostras2 = len(dados2count['U1']) +len(dados2count['U2']) + len(dados2count['U3'])
print(f'J2 tem {amostras2} dados')

J2 tem 10839 dados


In [13]:
dados3count = dados3.drop(['entry_id','acelX','acelY','acelZ','temperatura','tempo','ntemperatura','AceleraçãoX','AceleraçãoY','AceleraçãoZ'], axis = 1)
amostras3 = len(dados3count['U1']) +len(dados3count['U2']) + len(dados3count['U3'])
print(f'J3 tem {amostras3} dados')

J3 tem 10767 dados


In [14]:
total = amostras1+amostras2+amostras3
print(f'tiveram {total} envios de dados')

tiveram 32904 envios de dados


#### dados nulos

##### J1

In [15]:
dados1

,created_at,entry_id,acelX,acelY,acelZ,temperatura,U1,U2,U3,tempo,ntemperatura,AceleraçãoX,AceleraçãoY,AceleraçãoZ
0,2022-11-30T10:51:50-03:00,1,10796,-11048,5704,2684,514,434,826,2022-11-30 10:51:50-03:00,26.84,6.464158,-6.615044,3.415298
1,2022-11-30T10:52:53-03:00,2,10804,-10984,5676,2768,613,498,827,2022-11-30 10:52:53-03:00,27.68,6.468948,-6.576724,3.398533
2,2022-11-30T10:53:55-03:00,3,10748,-11112,5596,2811,0,104,828,2022-11-30 10:53:55-03:00,28.11,6.435417,-6.653364,3.350632
3,2022-11-30T10:54:58-03:00,4,3472,-15724,-6132,2834,0,102,828,2022-11-30 10:54:58-03:00,28.34,2.078877,-9.414822,-3.671565
4,2022-11-30T10:56:00-03:00,5,3068,-15636,-6072,2848,0,103,828,2022-11-30 10:56:00-03:00,28.48,1.836980,-9.362131,-3.635640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,2022-12-14T07:46:32-03:00,3762,0,0,0,0,0,0,0,2022-12-14 07:46:32-03:00,0.00,0.000000,0.000000,0.000000
3762,2022-12-14T07:58:25-03:00,3763,0,0,0,0,0,0,0,2022-12-14 07:58:25-03:00,0.00,0.000000,0.000000,0.000000
3763,2022-12-14T08:02:18-03:00,3764,0,0,0,0,0,0,0,2022-12-14 08:02:18-03:00,0.00,0.000000,0.000000,0.000000
3764,2022-12-14T08:06:14-03:00,3765,0,0,0,0,0,0,0,2022-12-14 08:06:14-03:00,0.00,0.000000,0.000000,0.000000


In [16]:
j1u1zero = dados1.query('U1 == 0')
#j1u1zero
print(f' Sensor U1 de J1 teve {len(j1u1zero)} dados nulos')

 Sensor U1 de J1 teve 3333 dados nulos


In [17]:
j1u2zero = dados1.query('U2 == 0')
#j1u2zero
print(f' Sensor U2 de J1 teve {len(j1u2zero)} dados nulos')

 Sensor U2 de J1 teve 3318 dados nulos


In [18]:
j1u3zero = dados1.query('U3 == 0')
#j1u3zero
print(f' Sensor U3 de J1 teve {len(j1u3zero)} dados nulos')

 Sensor U3 de J1 teve 3322 dados nulos


##### J2

In [19]:
dados2

,created_at,entry_id,acelX,acelY,acelZ,temperatura,U1,U2,U3,tempo,ntemperatura,AceleraçãoX,AceleraçãoY,AceleraçãoZ
0,2022-11-30T13:52:07-03:00,1,0,0,0,0,0,0,0,2022-11-30 13:52:07-03:00,0.0,0.0,0.0,0.0
1,2022-11-30T13:56:00-03:00,2,0,0,0,0,0,0,0,2022-11-30 13:56:00-03:00,0.0,0.0,0.0,0.0
2,2022-11-30T13:59:48-03:00,3,0,0,0,0,0,0,0,2022-11-30 13:59:48-03:00,0.0,0.0,0.0,0.0
3,2022-11-30T14:03:30-03:00,4,0,0,0,0,0,0,0,2022-11-30 14:03:30-03:00,0.0,0.0,0.0,0.0
4,2022-11-30T14:08:43-03:00,5,0,0,0,0,0,0,0,2022-11-30 14:08:43-03:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3608,2022-12-14T07:29:28-03:00,3609,0,0,0,0,0,0,0,2022-12-14 07:29:28-03:00,0.0,0.0,0.0,0.0
3609,2022-12-14T07:37:19-03:00,3610,0,0,0,0,0,0,0,2022-12-14 07:37:19-03:00,0.0,0.0,0.0,0.0
3610,2022-12-14T07:53:02-03:00,3611,0,0,0,0,0,0,0,2022-12-14 07:53:02-03:00,0.0,0.0,0.0,0.0
3611,2022-12-14T07:57:03-03:00,3612,0,0,0,0,0,0,0,2022-12-14 07:57:03-03:00,0.0,0.0,0.0,0.0


In [20]:
j2u1zero = dados2.query('U1 == 0')
#j2u1zero
print(f' Sensor U1 de J2 teve {len(j2u1zero)} dados nulos')

 Sensor U1 de J2 teve 3164 dados nulos


In [21]:
j2u2zero = dados2.query('U2 == 0')
#j2u2zero
print(f' Sensor U2 de J2 teve {len(j2u2zero)} dados nulos')

 Sensor U2 de J2 teve 3171 dados nulos


In [22]:
j2u3zero = dados2.query('U3 == 0')
#j2u3zero
print(f' Sensor U3 de J2 teve {len(j2u3zero)} dados nulos')

 Sensor U3 de J2 teve 3170 dados nulos


##### J3

In [23]:
dados3

,created_at,entry_id,acelX,acelY,acelZ,temperatura,U1,U2,U3,tempo,ntemperatura,AceleraçãoX,AceleraçãoY,AceleraçãoZ
0,2022-11-30T13:40:48-03:00,1,0,0,0,0,0,0,0,2022-11-30 13:40:48-03:00,0.00,0.000000,0.000000,0.000000
1,2022-11-30T13:44:31-03:00,2,0,0,0,0,0,0,0,2022-11-30 13:44:31-03:00,0.00,0.000000,0.000000,0.000000
2,2022-11-30T13:50:53-03:00,3,0,0,0,0,0,0,0,2022-11-30 13:50:53-03:00,0.00,0.000000,0.000000,0.000000
3,2022-11-30T13:54:35-03:00,4,15144,6348,-1388,2566,277,0,105,2022-11-30 13:54:35-03:00,25.66,9.067544,3.800896,-0.831072
4,2022-11-30T13:58:27-03:00,5,15412,1740,5716,2561,267,0,102,2022-11-30 13:58:27-03:00,25.61,9.228010,1.041833,3.422483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,2022-12-14T07:47:53-03:00,3585,0,0,0,0,0,0,0,2022-12-14 07:47:53-03:00,0.00,0.000000,0.000000,0.000000
3585,2022-12-14T07:55:47-03:00,3586,0,0,0,0,0,0,0,2022-12-14 07:55:47-03:00,0.00,0.000000,0.000000,0.000000
3586,2022-12-14T07:59:39-03:00,3587,0,0,0,0,0,0,0,2022-12-14 07:59:39-03:00,0.00,0.000000,0.000000,0.000000
3587,2022-12-14T08:03:39-03:00,3588,0,0,0,0,0,0,0,2022-12-14 08:03:39-03:00,0.00,0.000000,0.000000,0.000000


In [24]:
j3u1zero = dados3.query('U1 == 0')
#j3u1zero
print(f' Sensor U1 de J3 teve {len(j3u1zero)} dados nulos')

 Sensor U1 de J3 teve 3188 dados nulos


In [25]:
j3u2zero = dados3.query('U2 == 0')
#j3u2zero
print(f' Sensor U2 de J3 teve {len(j3u2zero)} dados nulos')

 Sensor U2 de J3 teve 3191 dados nulos


In [26]:
j3u3zero = dados3.query('U3 == 0')
#j3u3zero
print(f' Sensor U3 de J3 teve {len(j3u3zero)} dados nulos')

 Sensor U3 de J3 teve 3199 dados nulos


##### Graficos

In [27]:
fig3 = go.Figure()

In [28]:
# #J1
# fig3.add_trace(go.Scatter(x=j1u1zero['created_at'], y=j1u1zero['U1'], mode='markers',name='J1_U1'))
# fig3.add_trace(go.Scatter(x=j1u2zero['created_at'], y=j1u2zero['U2'], mode='markers',name='J1_U2'))
# fig3.add_trace(go.Scatter(x=j1u3zero['created_at'], y=j1u3zero['U3'], mode='markers',name='J1_U3'))
# #J2
# fig3.add_trace(go.Scatter(x=j2u1zero['created_at'], y=j2u1zero['U1'], mode='markers',name='J2_U2'))
# fig3.add_trace(go.Scatter(x=j2u2zero['created_at'], y=j1u2zero['U2'], mode='markers',name='J2_U2'))
# fig3.add_trace(go.Scatter(x=j2u3zero['created_at'], y=j2u3zero['U3'], mode='markers',name='J2_U3'))
# #J3
# fig3.add_trace(go.Scatter(x=j3u1zero['created_at'], y=j3u1zero['U1'], mode='markers',name='J3_U1'))
# fig3.add_trace(go.Scatter(x=j3u2zero['created_at'], y=j3u2zero['U2'], mode='markers',name='J3_U2'))
# fig3.add_trace(go.Scatter(x=j3u3zero['created_at'], y=j3u3zero['U3'], mode='markers',name='J3_U3'))

#### Envio de dados e Tempo sem internet

In [29]:
from datetime import datetime
from datetime import timedelta
import numpy as np

##### J1

In [30]:
#dados1

In [31]:
dias = [0]
horas = [0]
minutos = [0]
segundos = [0]

for i in pd.to_datetime(dados1['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  dias.append(i.day)
  horas.append(i.hour)
  minutos.append(i.minute)
  segundos.append(i.second)


In [32]:
tempo =[]
delta=[]

#for i,j in enumerate(pd.to_datetime(dados1['created_at'])):


#delta.remove(delta[0])

In [33]:
dados1['DifMinutos'] = delta[0:]      # coloca as informações de tempo no DataFrame

ValueError: ignored

In [ ]:
dados1['DifMinutos'][0] = np.nan     # tira a primeira informação de tempo (pois a diferença de minutos é feita entre duas datas e a primeira diferença foi feita com um dado nulo)

In [ ]:
dados1[['created_at','DifMinutos']].describe()

In [ ]:
#dadosj1 = pd.DataFrame(data = delta).rename({0 : 'Minutos'}, axis=1)
dados1media = str(dados1['DifMinutos'] .mean())
dados1media

###### Tempo sem Internet

In [ ]:
# analísa o tempo (acima da media) que o sensor nao enviou dados . Tempos sem envio de dados abaixo da média não são enxergados aqui

In [ ]:
sem_internet1 = dados1.query(f'DifMinutos > {dados1media}')

In [ ]:
sem_internet1[['created_at','DifMinutos']]

In [ ]:
sem_internet1['DifMinutos'].mean()


In [ ]:
figj1 = go.Figure()

In [ ]:
figj1.add_trace(go.Scatter(x=sem_internet1['created_at'], y = sem_internet1['DifMinutos'], mode= 'markers'))
                           

##### J2

In [ ]:
#dados2

In [ ]:
dias = [0]
horas = [0]
minutos = [0]
segundos = [0]

for i in pd.to_datetime(dados2['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  dias.append(i.day)
  horas.append(i.hour)
  minutos.append(i.minute)
  segundos.append(i.second)

In [ ]:
dif =[]
delta=[]

for i,j in enumerate(pd.to_datetime(dados2['created_at'])):

  dif = j - timedelta(days = dias[i], hours = horas[i], minutes = minutos[i],seconds = segundos[i])
  tempo = (dif.hour)*60 + dif.minute + (dif.second)/60        # faz a conversão de hora e segundo para minuto
  delta.append(tempo)    # salva as informações de tempo (em minutos)


#delta.remove(delta[0])

In [ ]:
dados2['DifMinutos'] = delta[0:]      # coloca as informações de tempo no DataFrame

In [ ]:
dados2['DifMinutos'][0] = np.nan     # tira a primeira informação de tempo (pois a diferença de minutos é feita entre duas datas e a primeira diferença foi feita com um dado nulo)

In [ ]:
dados2[['created_at','DifMinutos']]

In [ ]:
dados2[['created_at','DifMinutos']].describe()

In [ ]:
#dadosj2 = pd.DataFrame(data = delta).rename({0 : 'Minutos'}, axis=1)
dados2media = str(dados2['DifMinutos'] .mean())
dados2media

###### Tempo sem Internet

In [ ]:
# analísa o tempo (acima da media) que o sensor nao enviou dados . Tempos sem envio de dados abaixo da média não são enxergados aqui

In [ ]:
sem_internet2 = dados2.query(f'DifMinutos > {dados2media}')

In [ ]:
sem_internet2[['created_at','DifMinutos']]

In [ ]:
sem_internet2['DifMinutos'].mean()

In [ ]:
figj2 = go.Figure()

In [ ]:
figj2.add_trace(go.Scatter(x=sem_internet2['created_at'], y = sem_internet2['DifMinutos'], mode= 'markers'))
                           

##### J3

In [ ]:
#dados3

In [ ]:
dias = [0]
horas = [0]
minutos = [0]
segundos = [0]

for i in pd.to_datetime(dados3['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  dias.append(i.day)
  horas.append(i.hour)
  minutos.append(i.minute)
  segundos.append(i.second)

In [ ]:
dif =[]
delta=[]

for i,j in enumerate(pd.to_datetime(dados3['created_at'])):

  dif = j - timedelta(days = dias[i], hours = horas[i], minutes = minutos[i], seconds = segundos[i])
  tempo = (dif.hour)*60 + dif.minute + (dif.second)/60        # faz a conversão de hora e segundo para minuto
  delta.append(tempo)    # salva as informações de tempo (em minutos)

#delta.remove(delta[0])

In [ ]:
dados3['DifMinutos'] = delta[0:]      # coloca as informações de tempo no DataFrame

In [ ]:
dados3['DifMinutos'][0] = np.nan     # tira a primeira informação de tempo (pois a diferença de minutos é feita entre duas datas e a primeira diferença foi feita com um dado nulo)

In [ ]:
dados3[['created_at','DifMinutos']]

In [ ]:
dados3[['created_at','DifMinutos']].describe()

In [ ]:
#dadosj3 = pd.DataFrame(data = delta).rename({0 : 'Minutos'}, axis=1)
dados3media = str(dados3['DifMinutos'] .mean())
dados3media

###### Tempo sem Internet

In [ ]:
# analísa o tempo (acima da media) que o sensor nao enviou dados . Tempos sem envio de dados abaixo da média não são enxergados aqui

In [ ]:
sem_internet3 = dados3.query(f'DifMinutos > {dados3media}')

In [ ]:
sem_internet3[['created_at','DifMinutos']]

In [ ]:
sem_internet3['DifMinutos'].mean()

In [ ]:
figj3 = go.Figure()

In [ ]:
figj3.add_trace(go.Scatter(x=sem_internet3['created_at'], y = sem_internet3['DifMinutos'], mode= 'markers'))
                           

##### Todos os canais

In [ ]:
dados1.describe()
#dados1[['created_at','DifMinutos']].describe()

In [ ]:
dados2.describe()
#dados2[['created_at','DifMinutos']].describe()

In [ ]:
dados3.describe()
#dados3[['created_at','DifMinutos']].describe()

In [ ]:
# Quantidades de vezes que o tempo de envio de dados foi acima da média . Tempos sem envio de dados abaixo da média não são enxergados aqui

print(len(sem_internet1),len(sem_internet2),len(sem_internet3))

In [ ]:
figfinal = go.Figure()

In [ ]:
# Gráfico da quantidade de vezes que o tempo de envio de dados foi acima da média 
figfinal.add_trace(go.Scatter(x=sem_internet1['created_at'], y = sem_internet1['DifMinutos'], mode= 'markers', name = 'J1'))
figfinal.add_trace(go.Scatter(x=sem_internet2['created_at'], y = sem_internet2['DifMinutos'], mode= 'markers', name = 'J2'))
figfinal.add_trace(go.Scatter(x=sem_internet3['created_at'], y = sem_internet3['DifMinutos'], mode= 'markers', name = 'J3'))


#### Linha de Tendência

In [10]:
from datetime import datetime
from datetime import timedelta
import numpy as np

##### J1

In [11]:
dados1lt = dados1[['created_at','U1','U2','U3']]
dados1lt


,created_at,U1,U2,U3
0,2022-11-30T10:51:50-03:00,514,434,826
1,2022-11-30T10:52:53-03:00,613,498,827
2,2022-11-30T10:53:55-03:00,0,104,828
3,2022-11-30T10:54:58-03:00,0,102,828
4,2022-11-30T10:56:00-03:00,0,103,828
...,...,...,...,...
3761,2022-12-14T07:46:32-03:00,0,0,0
3762,2022-12-14T07:58:25-03:00,0,0,0
3763,2022-12-14T08:02:18-03:00,0,0,0
3764,2022-12-14T08:06:14-03:00,0,0,0


##### J2

In [12]:
dados2lt =  dados2[['created_at','U1','U2','U3']]
dados2lt

,created_at,U1,U2,U3
0,2022-11-30T13:52:07-03:00,0,0,0
1,2022-11-30T13:56:00-03:00,0,0,0
2,2022-11-30T13:59:48-03:00,0,0,0
3,2022-11-30T14:03:30-03:00,0,0,0
4,2022-11-30T14:08:43-03:00,0,0,0
...,...,...,...,...
3608,2022-12-14T07:29:28-03:00,0,0,0
3609,2022-12-14T07:37:19-03:00,0,0,0
3610,2022-12-14T07:53:02-03:00,0,0,0
3611,2022-12-14T07:57:03-03:00,0,0,0


##### J3

In [13]:
dados3lt =  dados3[['created_at','U1','U2','U3']]
dados3lt

,created_at,U1,U2,U3
0,2022-11-30T13:40:48-03:00,0,0,0
1,2022-11-30T13:44:31-03:00,0,0,0
2,2022-11-30T13:50:53-03:00,0,0,0
3,2022-11-30T13:54:35-03:00,277,0,105
4,2022-11-30T13:58:27-03:00,267,0,102
...,...,...,...,...
3584,2022-12-14T07:47:53-03:00,0,0,0
3585,2022-12-14T07:55:47-03:00,0,0,0
3586,2022-12-14T07:59:39-03:00,0,0,0
3587,2022-12-14T08:03:39-03:00,0,0,0


In [14]:
# data = pd.concat([dados1['created_at'],dados2['created_at'],dados3['created_at']], axis = 1)
# datam = pd.to_datetime(data)
# mediaU1 = pd.concat([dados1[['created_at','U1']],dados2[['created_at','U1']],dados3[['created_at','U1']]], axis = 1)
# display(mediaU1)


##### Tempo

In [16]:
# como os dados são enviados em tempos diferentes, aqui encontra-se um tempo médio para cada envio de dados dos 3 canais
# https://www.digitalocean.com/community/tutorials/python-string-to-datetime-strptime
# https://www.geeksforgeeks.org/python-datetime-strptime-function/

In [ ]:
# transforma a colua created_at em objetos datetime
dados1lt['created_at'] = pd.to_datetime(dados1lt['created_at'])
dados2lt['created_at'] = pd.to_datetime(dados2lt['created_at'])
dados3lt['created_at'] = pd.to_datetime(dados2lt['created_at'])

In [114]:
dadostempo1,dadostempo2, dadostempo3 = pd.DataFrame(data = [dados1lt['created_at']]).transpose(), pd.DataFrame(data = [dados2lt['created_at']]).transpose(), pd.DataFrame(data = [dados3lt['created_at']]).transpose() 


In [ ]:
dadostempo2

###### Completando tamanho das linhas

In [116]:
len(dadostempo1.index),len(dadostempo2.index),len(dadostempo3.index)

(3766, 3613, 3589)

In [111]:
# j2
for i in dadostempo1.index:
#  if dadostempo2['created_at'][i].day == dadostempo1['created_at'][i].day:  # verifica se a data (dia) do sensor j2 é diferente do sensor j1 
  if dadostempo2['created_at'][i].day == dadostempo1['created_at'][i].day:  # verifica se a data (dia) do sensor j2 é diferente do sensor j1 
    Ellipsis  
    ...

  else:
    if dadostempo2['created_at'][i] > dadostempo1['created_at'][i]: # verifica se a data (dia) do sensor j2 é maior que a do sensor j1
      corte = dadostempo2.iloc[i:]   # salva os dados seguintes do sensor j2 em um objeto
      dadostempo2 = dadostempo2.drop(index = dadostempo2.index[i+1:])  # dropa as linhas seguintes à i+1 e salva as linhas anteriores à i (para depois juntar com corte)  # https://towardsdatascience.com/delete-rows-and-columns-from-a-dataframe-using-pandas-drop-d2533cf7b4bd#:~:text=Delete%20rows%20and%20columns%20from%20a%20DataFrame%20using%20Pandas%20drop(),-9%20tricks%20to&text=Delete%20one%20or%20many%20rows,is%20the%20drop()%20method.

      dadostempo2['created_at'][i] = dadostempo1['created_at'][i]
      
      dt2 = pd.concat([dadostempo2,corte], axis = 0, ignore_index = True)

      dadostempo2 = dt2

    elif dadostempo2['created_at'][i] < dadostempo1['created_at'][i]: # verifica se a data (dia) do sensor j2 é maior que a do sensor j1
      corte = dadostempo1.iloc[i:]   # salva os dados seguintes do sensor j2 em um objeto
      dadostempo1 = dadostempo1.drop(index = dadostempo1.index[i+1:])  # dropa as linhas seguintes à i+1 e salva as linhas anteriores à i (para depois juntar com corte)  # https://towardsdatascience.com/delete-rows-and-columns-from-a-dataframe-using-pandas-drop-d2533cf7b4bd#:~:text=Delete%20rows%20and%20columns%20from%20a%20DataFrame%20using%20Pandas%20drop(),-9%20tricks%20to&text=Delete%20one%20or%20many%20rows,is%20the%20drop()%20method.

      dadostempo1['created_at'][i] = dadostempo2['created_at'][i]
    
  #      dadostempo2.iloc[i-1]['created_at'] = dadostempo1.iloc[i]['created_at']  # copia o dado de tempo do sensor j1 e coloca nessa linha i

      #dadostempo2.iloc[i]['created_at'] = dadostempo1.iloc[i]['created_at']  # copia o dado de tempo do sensor j1 e coloca nessa linha i
      #linha_extra =  dadostempo1['created_at'][i]
      # dadostempo2.iloc[i]['U1'], dadostempo2.iloc[i]['U2'], dadostempo2.iloc[i]['U3'] = np.nan , np.nan, np.nan  # preenche a linha i com valores nan
      
      dt1 = pd.concat([dadostempo1,corte], axis = 0, ignore_index = True)

      dadostempo1 = dt1
 #   corte = []
    

    

In [113]:
# j3
for i in dadostempo1.index:

  if dadostempo3['created_at'][i].day == dadostempo1['created_at'][i].day:  # verifica se a data (dia) do sensor j3 é diferente do sensor j1 
    Ellipsis  
    ...
    
  else:
    if dadostempo3['created_at'][i] > dadostempo1['created_at'][i]: # verifica se a data (dia) do sensor j2 é maior que a do sensor j1
      corte = dadostempo3.iloc[i:]   # salva os dados seguintes do sensor j2 em um objeto
      dadostempo3 = dadostempo3.drop(index = dadostempo3.index[i+1:])  # dropa as linhas seguintes à i+1 e salva as linhas anteriores à i (para depois juntar com corte)  # https://towardsdatascience.com/delete-rows-and-columns-from-a-dataframe-using-pandas-drop-d2533cf7b4bd#:~:text=Delete%20rows%20and%20columns%20from%20a%20DataFrame%20using%20Pandas%20drop(),-9%20tricks%20to&text=Delete%20one%20or%20many%20rows,is%20the%20drop()%20method.

      dadostempo3['created_at'][i] = dadostempo1['created_at'][i]
  
      dt3 = pd.concat([dadostempo3,corte], axis = 0, ignore_index = True)

      dadostempo3 = dt3

    elif dadostempo3['created_at'][i] < dadostempo1['created_at'][i]: # verifica se a data (dia) do sensor j2 é maior que a do sensor j1
      corte = dadostempo1.iloc[i:]   # salva os dados seguintes do sensor j2 em um objeto
      dadostempo1 = dadostempo1.drop(index = dadostempo1.index[i+1:])  # dropa as linhas seguintes à i+1 e salva as linhas anteriores à i (para depois juntar com corte)  # https://towardsdatascience.com/delete-rows-and-columns-from-a-dataframe-using-pandas-drop-d2533cf7b4bd#:~:text=Delete%20rows%20and%20columns%20from%20a%20DataFrame%20using%20Pandas%20drop(),-9%20tricks%20to&text=Delete%20one%20or%20many%20rows,is%20the%20drop()%20method.

      dadostempo1['created_at'][i] = dadostempo3['created_at'][i]
    

      # dadostempo2.iloc[i]['U1'], dadostempo2.iloc[i]['U2'], dadostempo2.iloc[i]['U3'] = np.nan , np.nan, np.nan  # preenche a linha i com valores nan
      
      dt1 = pd.concat([dadostempo1,corte], axis = 0, ignore_index = True)

      dadostempo1 = dt1

    

    

KeyError: ignored

In [95]:
dadostempo2['created_at'][30] , dadostempo1['created_at'][31]

(Timestamp('2022-11-30 11:46:13-0300', tz='pytz.FixedOffset(-180)'),
 Timestamp('2022-11-30 11:47:28-0300', tz='pytz.FixedOffset(-180)'))

In [98]:
for i in range(3762):
  if dadostempo2['created_at'][i].day == dadostempo1['created_at'][i].day:
    #print('ok')
    Ellipsis
    ...
  else:
    print('nao',i)

In [99]:
dadostempo2

,created_at
0,2022-11-30 13:52:07-03:00
1,2022-11-30 13:56:00-03:00
2,2022-11-30 13:59:48-03:00
3,2022-11-30 14:03:30-03:00
4,2022-11-30 14:08:43-03:00
...,...
3801,2022-12-14 07:29:28-03:00
3802,2022-12-14 07:37:19-03:00
3803,2022-12-14 07:53:02-03:00
3804,2022-12-14 07:57:03-03:00


In [93]:
lista = pd.DataFrame(data = [0,1,2,3,4,5,6,7,8,9,10,11,12])
lista = lista.drop(index = lista.index[10:])
lista

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [70]:
lista[0][7] = 8

In [71]:
lista

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6


In [48]:
lista

,0
3,3
4,4
5,5
6,6
7,7


In [ ]:
dados2lt.merge(dados1lt, left_on='created_at', right_on='created_at', how='outer')

In [ ]:
dados2lt

In [ ]:
ano1 = [0]
mes1 = [0]
dias1 = [0]
horas1 = [0]
minutos1 = [0]
segundos1 = [0]

for i in pd.to_datetime(dados1lt['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  ano1.append(i.year)
  mes1.append(i.month)
  dias1.append(i.day)
  horas1.append(i.hour)
  minutos1.append(i.minute)
  segundos1.append(i.second)

ano2 = [0]
mes2 = [0]
dias2 = [0]
horas2 = [0]
minutos2 = [0]
segundos2 = [0]

for i in pd.to_datetime(dados2lt['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  ano2.append(i.year)
  mes2.append(i.month) 
  dias2.append(i.day)
  horas2.append(i.hour)
  minutos2.append(i.minute)
  segundos2.append(i.second)

ano3 = [0]
mes3 = [0]
dias3 = [0]
horas3 = [0]
minutos3 = [0]
segundos3 = [0]

for i in pd.to_datetime(dados1lt['created_at']):  #pega as informações de tempo do dado original e salva em objetos de data
  ano3.append(i.year)  
  mes3.append(i.month)
  dias3.append(i.day)
  horas3.append(i.hour)
  minutos3.append(i.minute)
  segundos3.append(i.second)


In [ ]:
anos_medio = pd.DataFrame(data = [ano1,ano2,ano3]).mean()
#ano_medio

In [ ]:
mes_medio = pd.DataFrame(data = [mes1,mes2,mes3]).mean()
#mes_medio

In [ ]:
dias_medio = pd.DataFrame(data = [dias1,dias2,dias3]).mean()
#dias_medio

In [ ]:
dias_medio[59]

In [ ]:
horas_medio = pd.DataFrame(data = [horas1,horas2,horas3]).mean()
#horas_medio

In [ ]:
minutos_medio = pd.DataFrame(data = [minutos1,minutos2,minutos3]).mean()
#minutos_medio

In [ ]:
segundos_medio = pd.DataFrame(data = [segundos1,segundos2,segundos3]).mean()
#segundos_medio

In [ ]:
tempo = pd.DataFrame(data = [anos_medio,dias_medio,mes_medio,horas_medio,minutos_medio,segundos_medio]).transpose().rename({0:'Ano', 1:'Dia',2:'Mes',3:'Hora',4:'Minuto',5:'Segundo'}, axis = 1).drop(0,axis=0)
tempo

In [ ]:
format = '%Y %m %d %H %M %S'

In [ ]:
Tempo = [] 
for i in tempo.index:
  Tempo.append(str(int(tempo['Ano'][i])) + ' ' + str(int(tempo['Mes'][i])) + ' ' + str(int(tempo['Dia'][i])) + ' ' + str(int(tempo['Hora'][i])) + ' ' + str(int(tempo['Minuto'][i])) + ' ' + str(int(tempo['Segundo'][i])))
  #Tempo.append(str(tempo['Ano'][i]) + ' ' + str(tempo['Mes'][i]) + ' ' + str(tempo['Dia'][i]) + ' ' + str(tempo['Hora'][i]) + ' ' + str(tempo['Minuto'][i]) + ' ' + str(int(tempo['Segundo'][i])))


In [ ]:
horario = []
for i in Tempo:
  horario.append(datetime.strptime(i,format))

In [ ]:
t = pd.DataFrame(data = horario).rename({0:'Horário'},axis = 1)
t

##### Linha de tendencia (finalmente rs)

In [ ]:
# https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html

In [ ]:
j1u1 = dados1lt['U1']#.to_numpy()
j1u2 = dados1lt['U2']#.to_numpy()
j1u3 = dados1lt['U3']#.to_numpy() 

j2u1 = dados2lt['U1']#.to_numpy() 
j2u2 = dados2lt['U2']#.to_numpy() 
j2u3 = dados2lt['U3']#.to_numpy() 

j3u1 = dados3lt['U1']#.to_numpy() 
j3u2 = dados3lt['U2']#.to_numpy() 
j3u3 = dados3lt['U3']#.to_numpy() 

In [ ]:
mediaU1 = pd.DataFrame(data = (j1u1,j2u1,j3u1)).mean()
mediaU1

In [ ]:
mediaU2 = pd.DataFrame(data = (j1u2,j2u2,j3u2)).mean()
mediaU2

In [ ]:
mediaU3 = pd.DataFrame(data = (j1u3,j2u3,j3u3)).mean()
mediaU3

In [ ]:
dados = pd.DataFrame()
dados['Horário'] = t['Horário']
dados['U1'] = mediaU1
dados['U2'] = mediaU2
dados['U3'] = mediaU3
dados

In [ ]:
dados.query('U2 > 410.6667')

In [ ]:
figlt = go.Figure()

In [ ]:
figlt.add_trace(go.Scatter(x= dados['Horário'], y = dados['U1'], mode= 'markers', name = 'U1'))
figlt.add_trace(go.Scatter(x= dados['Horário'], y = dados['U2'], mode= 'markers', name = 'U2'))
figlt.add_trace(go.Scatter(x= dados['Horário'], y = dados['U3'], mode= 'markers', name = 'U3'))

In [ ]:
#ltU1 = np.polyfit(x = , y = (j1u1 , j2u1, j3u1), deg = 1)

In [ ]:
dados1lt[50:60]

In [ ]:
dados2lt[50:60]

#   

# Calculo dos angulos e das medias móveis

In [ ]:
#Calcula os ángulos nos eixos Y, X e Z  ,ainda que de forma lenta
dados["anX"]=np.degrees(np.arctan2(dados['acelY'],dados["acelZ"]))
dados["anY"]=np.degrees(np.arctan2(dados['acelX'],dados["acelZ"]))
dados["anZ"]=np.degrees(np.arctan2(dados['acelX'],dados["acelY"]))

#Ajusta os angulos para que todos sejam positivos...isso evita confusões visuais onde 180º  fica longe de -180º 
dados.loc[dados['anX'] < 0,'anX'] = (dados.loc[dados['anX'] < 0,'anX']) + 360
dados.loc[dados['anY'] < 0,'anY'] = (dados.loc[dados['anY'] < 0,'anY']) + 360
dados.loc[dados['anZ'] < 0,'anZ'] = (dados.loc[dados['anZ'] < 0,'anZ']) + 360
#for i in dados.index:         #varredura do atributo index (linhas) do dataframe
#    if dados.loc[i,'anX'] < 0:
#        dados.loc[i,'anX'] += 360 # equivalente a dados.loc[i,'anX'] = dados.loc[i,'anX'] + 360
#        
#    if dados.loc[i,'anY'] < 0:
#        dados.loc[i,'anY'] += 360
#        
#    if dados.loc[i,'anZ'] < 0:
#        dados.loc[i,'anZ']+= 360

#Limpeza de dados

dados['MedianX'] = dados['anX'].rolling(window=1000).mean()
dados['MedianY'] = dados['anY'].rolling(window=1000).mean()
dados['MedianZ'] = dados['anZ'].rolling(window=1000).mean()
dados['MDx'] = dados['MedianX'].diff()
dados['MDy'] = dados['MedianY'].diff()
dados['MDz'] = dados['MedianZ'].diff()

dados['axD'] = dados['anX'].diff()
dados['ayD'] = dados['anY'].diff()
dados['azD'] = dados['anZ'].diff()


C1= np.abs(dados['axD'])>50 
C2= np.abs(dados['MDx'])<5
dados = dados.drop(dados[ C1 & C2 ].index)

C1= np.abs(dados['ayD'])>50 
C2= np.abs(dados['MDy'])<5
dados = dados.drop(dados[ C1 & C2 ].index)

C1= np.abs(dados['azD'])>50 
C2= np.abs(dados['MDz'])<5
dados = dados.drop(dados[ C1 & C2 ].index)


In [ ]:
dados['MedianX']=dados['anX'].rolling(window=1000).mean()
dados['MedianY']=dados['anY'].rolling(window=1000).mean()
dados['MedianZ']=dados['anZ'].rolling(window=1000).mean()
display(dados)
print(dados.describe())

,created_at,entry_id,acelX,acelY,acelZ,temperatura,U1,U2,U3,tempo,...,anZ,MedianX,MedianY,MedianZ,MDx,MDy,MDz,axD,ayD,azD
0,2021-08-19T17:10:36-03:00,1,-1100,-604,-16388,29.94,1,1,0,2021-08-19 17:10:36-03:00,...,241.229212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-19T17:11:36-03:00,2,-1128,-596,-16580,30.13,1,1,0,2021-08-19 17:11:36-03:00,...,242.149425,NaN,NaN,NaN,NaN,NaN,NaN,-0.052031,0.051987,0.920213
2,2021-08-19T17:12:36-03:00,3,-1048,-564,-16460,30.04,1,1,0,2021-08-19 17:12:36-03:00,...,241.712287,NaN,NaN,NaN,NaN,NaN,NaN,-0.096255,-0.248974,-0.437138
3,2021-08-19T17:13:37-03:00,4,-820,-328,-16468,30.13,1,1,0,2021-08-19 17:13:37-03:00,...,248.198591,NaN,NaN,NaN,NaN,NaN,NaN,-0.821432,-0.792471,6.486304
4,2021-08-19T17:14:37-03:00,5,-876,-248,-16368,30.27,1,1,0,2021-08-19 17:14:37-03:00,...,254.192916,NaN,NaN,NaN,NaN,NaN,NaN,-0.272982,0.212888,5.994325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571076,2022-10-26T19:34:45-03:00,571077,880,-932,-16256,28.39,1,-1,0,2022-10-26 19:34:45-03:00,...,136.643799,183.059356,177.012993,135.733404,0.000119,-0.000401,-0.002903,0.050204,0.331322,3.357264
571077,2022-10-26T19:35:46-03:00,571078,1124,-920,-16180,28.44,1,-1,0,2022-10-26 19:35:46-03:00,...,129.300509,183.059675,177.012096,135.729055,0.000319,-0.000897,-0.004348,-0.026976,-0.875249,-7.343290
571078,2022-10-26T19:36:46-03:00,571079,1008,-912,-16112,28.53,1,-1,0,2022-10-26 19:36:46-03:00,...,132.137595,183.060003,177.011385,135.725773,0.000328,-0.000710,-0.003283,-0.014652,0.393990,2.837086
571079,2022-10-26T19:37:46-03:00,571080,936,-904,-16244,28.53,1,-1,0,2022-10-26 19:37:46-03:00,...,134.003652,183.059992,177.010846,135.720565,-0.000011,-0.000539,-0.005208,-0.054400,0.282067,1.866057


            entry_id          acelX          acelY          acelZ  \
count  571017.000000  571017.000000  571017.000000  571017.000000   
mean   285568.225690    -955.438819    -335.012492  -16214.792330   
std    164844.695816     316.572872     339.535644     136.734946   
min         1.000000   -6944.000000   -5156.000000  -23620.000000   
25%    142818.000000   -1040.000000    -452.000000  -16304.000000   
50%    285572.000000    -896.000000    -412.000000  -16228.000000   
75%    428326.000000    -784.000000    -364.000000  -16144.000000   
max    571081.000000    1132.000000    5408.000000  -14756.000000   

         temperatura             U1             U2             U3  \
count  571017.000000  571017.000000  571017.000000  571017.000000   
mean       27.868109       0.859339       0.992834      -0.003767   
std         2.150573       0.511407       0.119503       0.061403   
min        22.650000      -1.000000      -1.000000      -1.000000   
25%        26.410000       1.0000

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada

m3 = (dados['tempo']<start_date)

teste_dados = dados[m3]

fig2=px.line(teste_dados,x='tempo',y=['MedianX','MedianY','MedianZ'],title="Médias Móveis",template="plotly_white",markers=True,hover_name='temperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (Graus)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **TEMPERATURA COM DADOS COMPLETOS**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada

m3 = (dados['tempo']<start_date)

teste_dados = dados[m3]

fig2=px.line(teste_dados,x='tempo',y=['ntemperatura'],title="Temperatura",template="plotly_white",markers=True,hover_name='temperatura')
fig2['data'][0]['line']['color']='rgb(255,0,0)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (ºC)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **GRÁFICO DE ACELERAÇÃO**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados['tempo']<start_date)

teste_dados = dados[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['acelX','acelY','acelZ'],title="Acelerações (bits)",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (bits)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados['tempo']<start_date)

teste_dados = dados[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['anX','anY','anZ'],title="Ângulos",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (Graus)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados['tempo']<start_date)

teste_dados = dados[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['AceleraçãoX','AceleraçãoY','AceleraçãoZ'],title="Acelerações (m/s²)",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (m/s²)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **UMIDADES**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados['tempo']<start_date)

teste_dados = dados[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['U1','U2','U3'],title="Umidades",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(135,206,250)'
fig2['data'][1]['line']['color']='rgb(0,191,255)'
fig2['data'][2]['line']['color']='rgb(25,25,112)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (bits)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **GRÁFICOS COM TODOS OS DADOS**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados['tempo']<start_date)

teste_dados = dados[m1]
#Forma de plotar em ploty

fig2=px.line(teste_dados,x='tempo',y=['acelX','acelY','acelZ','ntemperatura','U1','U2','U3'],title="Teste em Campo - Estufa UFF",template="plotly_white",markers=True,hover_name='temperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'
fig2['data'][3]['line']['color']='rgb(255,0,0)'
fig2['data'][4]['line']['color']='rgb(135,206,250)'
fig2['data'][5]['line']['color']='rgb(0,191,255)'
fig2['data'][6]['line']['color']='rgb(25,25,112)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (bits)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **REDUZINDO O TAMANHO DOS DADOS**

In [ ]:
intervalo = 100  #De quantos em quantos dados serão utilizados do dataframe antigo
nindex = range(0,len(dados),intervalo)
dados_reduzido = dados.iloc[nindex,:]
print(dados_reduzido)

### **TEMPERATURA COM DADOS REDUZIDOS**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada

m3 = (dados_reduzido['tempo']<start_date)

teste_dados = dados_reduzido[m3]

fig2=px.line(teste_dados,x='tempo',y=['ntemperatura'],title="Temperatura",template="plotly_white",markers=True,hover_name='temperatura')
fig2['data'][0]['line']['color']='rgb(255,0,0)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (ºC)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **GRÁFICOS DE ACELERAÇÃO COM DADOS REDUZIDOS**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados_reduzido['tempo']<start_date)

teste_dados = dados_reduzido[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['anX','anY','anZ'],title="Ângulos",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (Graus)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados_reduzido['tempo']<start_date)

teste_dados = dados_reduzido[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['AceleraçãoX','AceleraçãoY','AceleraçãoZ'],title="Acelerações (m/s²)",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (m/s²)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **UMIDADE REDUZIDA**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados_reduzido['tempo']<start_date)

teste_dados = dados_reduzido[m1]
#Forma de plotar em ploty
fig2=px.line(teste_dados,x='tempo',y=['U1','U2','U3'],title="Umidades (com temperatura)",template="plotly_white",markers=True,hover_name='ntemperatura')
fig2['data'][0]['line']['color']='rgb(135,206,250)'
fig2['data'][1]['line']['color']='rgb(0,191,255)'
fig2['data'][2]['line']['color']='rgb(25,25,112)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (bits)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **DADOS TOTAIS REDUZIDOS**

In [ ]:
start_date = pd.to_datetime('2022-06-20 15:00:00 -03')  #definir a data desejada


m1 = (dados_reduzido['tempo']<start_date)

teste_dados = dados_reduzido[m1]
#Forma de plotar em ploty

fig2=px.line(teste_dados,x='tempo',y=['anX','anY','anZ','ntemperatura','U1','U2','U3'],title="Teste em Campo - Estufa UFF",template="plotly_white",markers=True,hover_name='temperatura')
fig2['data'][0]['line']['color']='rgb(148,0,211)'
fig2['data'][1]['line']['color']='rgb(255,140,0)'
fig2['data'][2]['line']['color']='rgb(3,187,133)'
fig2['data'][3]['line']['color']='rgb(255,0,0)'
fig2['data'][4]['line']['color']='rgb(135,206,250)'
fig2['data'][5]['line']['color']='rgb(0,191,255)'
fig2['data'][6]['line']['color']='rgb(25,25,112)'

#Códigos para melhorar o grafico
fig2.update_traces(line=dict(width=4)) #Define aqui a largura das linhas do grafico neste caso com largura 
fig2.update_layout(font_family="Courier New",font_size = 16) #Define aqui o tipo de Fonte e o tamanho
fig2.update_traces(patch={"line": { "dash": 'dot'}}, selector={"legendgroup": "limite"})  #Define aqui o tipo de linha para a linha do limite
fig2.update_layout(legend_title=dict(text = '')) #Coloca um texto explicando as legendas.. neste caso deixei sem
fig2.update_layout(xaxis_title='Tempo',  yaxis_title='Leitura Sensor (graus; ºC; bits)') #titulos dos eixos X e Y

#Configuração para deixar a opção de baixar a imagem com maior resolução.
config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',

    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig2.show(config=config)

### **ANÁLISE DOS DADOS**

In [ ]:
#SEM INCLINAÇÃO CONTROLADO RESIDÊNCIA
start_date = pd.to_datetime('2021-08-22 15:00:00 -03')  #definir a data desejada
end_date = pd.to_datetime('2021-09-18 15:00:00 -03')  #definir a data desejada
m1 = (dados['tempo']>start_date)
m2 = (dados['tempo']<end_date)
teste_dados = dados[m2&m1]
print("Dados antes da borracha")
display(teste_dados.describe())

#COM INCLINAÇÃO CONTROLADO RESIDÊNCIA
start_date = pd.to_datetime('2021-09-18 15:00:00 -03')  #definir a data desejada
end_date = pd.to_datetime('2021-09-26 15:00:00 -03')  #definir a data desejada
m1 = (dados['tempo']>start_date)
m2 = (dados['tempo']<end_date)
teste_dados = dados[m1&m2]
print("Dados depois da borracha")
display(teste_dados.describe())

#CONTROLADO UNIVERSIDADE
start_date = pd.to_datetime('2022-07-30 15:00:00 -03')  #definir a data desejada
end_date = pd.to_datetime('2022-10-23 23:00:00 -03')  #definir a data desejada
m1 = (dados['tempo']>start_date)
m2 = (dados['tempo']<end_date)
teste_dados = dados[m2&m1]   ##Linha modificada  O filtro só tinha m2.. tinha que ser m1 E m2... adicionei no código  
print("Dados teste controlado na universidade - departamento")
display(teste_dados.describe())

#EM CAMPO ESTUFA UNIVERSIDADE
start_date = pd.to_datetime('2022-03-22 00:00:00 -03')  #definir a data desejada
end_date = pd.to_datetime('2022-03-25 11:00:00 -03')  #definir a data desejada
m1 = (dados['tempo']>start_date)
m2 = (dados['tempo']<end_date)
teste_dados = dados[m2&m1]   ##Linha modificada  O filtro só tinha m2.. tinha que ser m1 E m2... adicionei no código  
print("Dados teste em campo - Estufa")
display(teste_dados.describe())